# Analysis of StackOverflow Survey. Part IV 

In this notebook we build a predictiv model for job satisfaction. 

In [1]:
# import neccessary packages and libraries
import os
from collections import defaultdict

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
# to render plots in the notebook
%matplotlib inline

import seaborn as sns
# set a theme for seaborn
sns.set_theme()

from sklearn.linear_model import LinearRegression

from sklearn import (
    ensemble,
    preprocessing,
    tree,
)
from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
)
from sklearn.metrics import (
    r2_score, 
    mean_squared_error,
    auc,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
)

In [2]:
# import local module containing the neccessary functions
import utils_functions as uf

# forces the interpreter to re-import the module
import importlib
importlib.reload(uf);

## State the question
I am addressing the third question in this notebook. What can we tell about the job satisfaction of a data coder? What factors do influence it? Also, predict the job satisfaction for a developer who works with big data. 

This is a classification question, we are predicting a satisfaction level for a data developer, which includes: data scientist or machine learning specialist, data or business analyst and data engineer.

## Performance metrics - to review at the end

The following performance measures will be used in this project:
1. Cross validation via StratifiedKFold with 10 folds.
2. Confusion matrix, in particular precision, recall and F1 score.
3. The ROC curve and the related AUC score.

## Gather the data

Upload the data and keep the subset that contains those developers that work in data science related fields.

In [48]:
# create a path string
mypath = os.getcwd()

# upload the datafiles as pandas dataframes
df1 = pd.read_csv(mypath+'/data/survey20_updated.csv')

# check the uploaded data
df1.shape

(64461, 25)

In [49]:
# the data frame that contains the data developers only
df1 = df1[df1.DevClass == 'data_coder']

# check the size of the data
df1.shape

(8726, 25)

In [50]:
# create a list of columns to be used in this analysis
list_cols = ['MainBranch', 'ConvertedComp', 'Country',
       'EdLevel', 'Employment',
       'JobSat', 'EdImpt',
       'Learn', 'Overtime', 'OpSys', 'OrgSize', 
       'UndergradMajor', 'WorkWeekHrs']

In [51]:
# the dataset that contains only the listed columns
df1 = df1[list_cols]
df1.shape

(8726, 13)

In [52]:
# reset the index 
df1.reset_index(drop=True, inplace=True)

In [53]:
# gather information on dtypes and missing values
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8726 entries, 0 to 8725
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   MainBranch      8699 non-null   object 
 1   ConvertedComp   5810 non-null   float64
 2   Country         8726 non-null   object 
 3   EdLevel         8581 non-null   object 
 4   Employment      8726 non-null   object 
 5   JobSat          8726 non-null   int64  
 6   EdImpt          8206 non-null   object 
 7   Learn           7979 non-null   object 
 8   Overtime        7424 non-null   object 
 9   OpSys           8120 non-null   object 
 10  OrgSize         7590 non-null   object 
 11  UndergradMajor  8053 non-null   object 
 12  WorkWeekHrs     6995 non-null   float64
dtypes: float64(2), int64(1), object(10)
memory usage: 886.4+ KB


## Data profiling

In [54]:
# run this once to generate a profiling report and save it as html file

#import pandas_profiling
#profile = pandas_profiling.ProfileReport(df, minimal=False)
#profile.to_file(output_file="data_coders_report.html")

## Data preprocessing 

### Remove duplicates

In [55]:
# drop duplicate rows, if any
df1.drop_duplicates(subset=None, keep='first', inplace=True)
df1.shape

(8687, 13)

### Create bins for the WorkWeekHrs column

In [56]:
# create the labels
cut_labels = ['<10', '10-20', '20-30', '30-40', '40-50', '>50']

# define the bins 
m = df1.WorkWeekHrs.max()
cut_bins = [0, 10, 20, 30, 40, 50, m]

# create a new column which contains the new labels
df1['WorkWeek_Bins'] = pd.cut(df1['WorkWeekHrs'], bins=cut_bins, labels=cut_labels)

# check for success
df1['WorkWeek_Bins'].value_counts()

30-40    3844
40-50    1836
>50       610
<10       284
20-30     276
10-20     140
Name: WorkWeek_Bins, dtype: int64

In [57]:
# count the missing values in the new column
df1['WorkWeek_Bins'].isnull().sum()

1697

In [58]:
# change the type of the newly created column
df1['WorkWeek_Bins'] = df1['WorkWeek_Bins'].astype('object')

In [60]:
# drop the WorkWeekHrs column
df1.drop(columns = 'WorkWeekHrs', inplace=True);

### Create bins for the ConvertedComp column

In [133]:
# we could use quantile, however I prefer custom bins here
cut_labels = ['<10K', '10K-30K', '30K-50K', '50K-100K', '100K-200K', '>200K']

# define the bins 
m = X_train.ConvertedComp.max()
cut_bins = [0, 10000, 30000, 50000, 100000, 200000, m]

# create a new column which contains the new labels
df1['Comp_Bins'] = pd.cut(df1['ConvertedComp'], bins=cut_bins, labels=cut_labels)

# change the type of the newly created column
df1['Comp_Bins'] = df1['Comp_Bins'].astype('object')

# drop the WorkWeekHrs column
df1.drop(columns = 'ConvertedComp', inplace=True);

## Create features and target

Create a dataframe (X) with the features and a pandas series (y) that contains the labels.

In [134]:
# create a copy of the pre-processed dataframe
df2 = df1.copy()

In [135]:
# create the predictors dataframe
X = df2.drop(columns = 'JobSat')

# create the labels
y = df2['JobSat']

# check for success
X.info(), len(y)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8687 entries, 0 to 8725
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   MainBranch      8660 non-null   object
 1   Country         8687 non-null   object
 2   EdLevel         8542 non-null   object
 3   Employment      8687 non-null   object
 4   EdImpt          8194 non-null   object
 5   Learn           7944 non-null   object
 6   Overtime        7419 non-null   object
 7   OpSys           8083 non-null   object
 8   OrgSize         7584 non-null   object
 9   UndergradMajor  8016 non-null   object
 10  WorkWeek_Bins   6990 non-null   object
 11  Comp_Bins       5783 non-null   object
dtypes: object(12)
memory usage: 882.3+ KB


(None, 8687)

## Sample data

We will use $30 \%$ data for testing:

In [136]:
# split the data in train and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [137]:
# check for success
X_train.shape, len(y_train), X_test.shape, len(y_test)

((6080, 12), 6080, (2607, 12), 2607)

## Impute missing values in ConvertedCompensation column

Now that we have test and train data, we can impute missing values on the training set, and use the trained imputers to fill in the test dataset.

## Transform categorical data 

In [ ]:
# check the dtypes and 

In [82]:
# create a list of categorical features
cat_cols = df.select_dtypes(include=['object']).copy().columns
non_cat = ['ConvertedComp', 'WorkWeekHrs']

In [83]:
# encode the categorical variablesas dummy, drop the first column for each feature
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

In [84]:
df.shape


(8722, 419)